In [4]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, binom
import os
import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.discovery import build
import json
import tweepy

In [ ]:
class Graph():
    '''
    A class to generate graphs.
    '''

    def __init__(self):
        '''
        Creates and empty graph.
        '''
        
        self.G = nx.Graph()

    def generateGraph(self, N, p, graphType, k = 1, m = 1, showGraph = False):
        '''
        Generates a chosen graph with N nodes and edges with probability p.
        Args:
        N - number of nodes
        p - probability of connection between two chosen nodes
        k - number of neighbors (used in Watts-Strogatz graph), the deafult is 1
        m - number of new links (used in Barabasi-Albert graph), the default is 1
        graphType - type of graph from: randomGraph using the Gilbert model, wattsStrogatz and barabasiAlbert
        showGraph - determines if graph is shown, the default is False
        '''
        
        if type(N) is not int:
            return 'N must be an integer'
        elif type(k) is not int:
            return 'k must be an integer'
        elif type(m) is not int:
            return 'm must be an integer'
        elif m > N:
            return 'N must be greater than m'
        elif N < 0:
            return 'N must be a positive number'
        elif p < 0 or p > 1:
            return 'p must be between 0 and 1'
        
        if graphType == 'randomGraph':
            for i in range(N):
                self.G.add_node(i)

            for i in range(N):
                for j in range(N):
                    if i != j:
                        q = np.random.rand()
                        if q < p:
                            self.G.add_edge(i, j)

            if showGraph == True:
                nx.draw(self.G, node_size = 10, width = 0.3, node_color='black', edge_color='gray')
                plt.show()
        
        elif graphType == 'wattsStrogatz':
            for i in range(N):
                self.G.add_node(i)
                
            for i in range(N):
                for j in range(1, k//2 + 1):
                    n = (i + j) % N
                    self.G.add_edge(i, n)
                    
            for i in range(N):
                for j in range(1, k//2 + 1):
                    if i != j:
                        q = np.random.rand()
                        if q < p:
                            n = (1 + j) % N
                            new_n = np.random.choice([i for i in range(N)], 1)[0]
                            while new_n == i or self.G.has_edge(i, new_n):
                                new_n = np.random.choice([i for i in range(N)], 1)[0]
                            if self.G.has_edge(i, n):
                                self.G.remove_edge(i, n)
                            self.G.add_edge(i, new_n)
                            
            if showGraph == True:
                nx.draw(self.G, node_size = 10, width = 0.3, node_color='black', edge_color='gray')
                plt.show()
        
        elif graphType == 'barabasiAlbert':
            self.G.add_nodes_from(np.arange(m))
            self.G.add_edges_from([(i, j) for i in range(m) for j in range(i + 1, m)])
            
            for i in range(m, N):
                d = [self.G.degree(j) for j in range(i)]
                p = d / np.sum(d)
                new_n = np.random.choice(np.arange(i), size = m, replace = False, p = p)
                self.G.add_node(i)
                self.G.add_edges_from(zip([i]*m, new_n))

            if showGraph == True:
                nx.draw(self.G, node_size = 10, width = 0.3, node_color='black', edge_color='gray')
                plt.show()
        
        else:
            return 'You must choose a type of graph from: randomGraph, wattsStrogatz or barabasiAlbert'
            
        return self.G
    
    def numberOfVerticesAndEdges(self):
        '''
        Prints number of nodes and edges in the graph.
        '''
        
        print(f"Nodes: {self.G.number_of_nodes()}, Edges: {self.G.number_of_edges()}")
        
    def averageDegree(self):
        """
        Calculates average degree of the nodes in the graph.
        """
        
        d = []
        for i in self.G.nodes:
            d.append(self.G.degree(i))
            
        print(f"Average degree is: {np.mean(d)}")
        return np.mean(d)


    def degreeDistributionVar(self):
        '''
        Calculates the variance of the degree distribution (binomial distribution) of a given graph.
        '''
        
        d = [d for n, d in self.G.degree()]
        
        print(f"Variance of a degree distribution: {np.var(d)}")
        return np.var(d)
    
    def frequencyPlot(self):
        '''
        Generates a frequency plot for the degree distribution of the given graph.
        '''
        
        histogram = nx.degree_histogram(self.G)
        ds = range(len(histogram))
        
        plt.bar(ds, histogram)
        plt.xlabel('# of Degrees')
        plt.ylabel('#')
        plt.title('Frequency plot for degrees')
        plt.show()
        
    def fitPoissonDistribution(self):
        '''
        Fits a Poisson distribution to the degree distribution of the given graph.
        '''
        
        d = [d for n, d in self.G.degree()]
        
        fittedModel = poisson(np.mean(d))
        
        histogram = nx.degree_histogram(self.G)
        ds = range(len(histogram))

        plt.bar(ds, histogram, label = 'Frequency plot for degrees')
        plt.plot(ds, len(d) * fittedModel.pmf(ds), 'r-', label = 'Fitted Poisson distribution')
        plt.xlabel('# of Degrees')
        plt.ylabel('#')
        plt.title('Fitted Poisson distribution for degree distribution')
        plt.legend()
        plt.show()
        
#     def fitBinomialDistribution(self):
#         '''
#         Fits a binomial distribution to the degree distribution of the given graph.
#         '''
        
#         d = [d for n, d in self.G.degree()]
        
#         p = np.mean(d) / (len(d) - 1)
        
#         fittedModel = binom(len(d) - 1, p)
        
#         histogram = nx.degree_histogram(self.G)
#         ds = range(len(histogram))
        
#         plt.bar(ds, histogram, label = 'Frequency plot for degrees')
#         plt.plot(range(max(d)+1), fittedModel.pmf(range(max(d)+1)), 'r-', label = 'Fitted Binomial distribution')
#         plt.xlabel('# of Degrees')
#         plt.ylabel('#')
#         plt.title('Fitted Binomial distribution for degree distribution')
#         plt.legend()
#         plt.show()

In [ ]:
G = Graph()

In [ ]:
G.generateGraph(10, 0.05, 'barabasiAlbert', showGraph = True, m = 2)

In [ ]:
G.numberOfVerticesAndEdges()

In [ ]:
G.averageDegree()

In [ ]:
G.degreeDistributionVar()

In [ ]:
G.frequencyPlot()

In [ ]:
G.fitPoissonDistribution()

In [ ]:
# Authentication keys
consumer_key = "your_consumer_key"
consumer_secret = "your_consumer_secret"
access_token = "your_access_token"
access_token_secret = "your_access_token_secret"

# Authenticate to Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create API object
api = tweepy.API(auth)

# Get user's followers
user = api.get_user("GordonRamsay")
followers = []
for follower in tweepy.Cursor(api.followers, screen_name=user.screen_name).items():
    followers.append(follower)

# Get followers of user's followers
edges = []
for follower in followers:
    print(f"Follower: {follower.screen_name}")
    for sub_follower in tweepy.Cursor(api.followers, screen_name=follower.screen_name).items():
        edge = (follower.screen_name, sub_follower.screen_name)
        edges.append(edge)
        print(f"\tSub follower: {sub_follower.screen_name}")
        
# Write results as edges to a file
with open("twitter_edges.txt", "w") as f:
    for edge in edges:
        f.write(f"{edge[0]}, {edge[1]}\n")